In [ ]:
from huggingface_hub import notebook_login
notebook_login()


LORA

In [ ]:
!pip install -q torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0
!pip install -q transformers==4.44.0 datasets==2.19.0 accelerate==0.34.0 trl==0.9.6 loralib evaluate==0.4.2 scipy==1.11.4 numpy==1.26.4 sentencepiece protobuf==4.25.3
!pip install -q numpy==1.26.4 scipy==1.11.4
# restrart runtime after this command from Runtime tab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 767.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176

In [ ]:
import os, math, torch, loralib as lora
import torch.nn as nn
from typing import Dict, Any, List
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments


# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"           # fits on Colab Pro GPU with small batch/accum
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-loralib-dolly"
RANK         = 16                             # LoRA rank
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Load tokenizer & base model
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Load model without device_map="auto" initially to ensure all weights are materialized on CPU.
# This prevents layers from being on a 'meta' device, allowing loralib to inject properly.
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    # Removed device_map="auto" here
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# Using "eager" attention implementation as recommended by earlier stderr for Gemma2 models
model.config.attn_implementation = "eager"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# -------------------


# --------------------
# Inject LoRA (loralib) by replacing target Linear layers
# --------------------
def _wrap_linear_with_lora(parent: nn.Module, child_name: str, child: nn.Module):
    # This check is now less likely to trigger if model is fully loaded to CPU
    if child.weight.device.type == "meta":
        print(f"⚠️ Skipping {child_name}: still on meta device.")
        return

    in_f, out_f = child.in_features, child.out_features
    new_layer = lora.Linear(
        in_features=in_f,
        out_features=out_f,
        r=RANK,
        bias=(child.bias is not None)
    )
    # Copy pretrained weights to preserve original functionality
    with torch.no_grad():
        new_layer.weight.copy_(child.weight.data)
        if child.bias is not None:
            new_layer.bias.copy_(child.bias.data)

    setattr(parent, child_name, new_layer)
TARGET_LINEAR_NAMES = {
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
}

def _inject_lora(module: nn.Module, prefix: str = ""):
    # For each child, if it's a Linear whose qualified name ends with any of TARGET_LINEAR_NAMES, wrap with lora.Linear
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(qname.endswith(tn) for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora(child, qname)

# actually inject
_inject_lora(model)

# --- Debug: check what Linear layers exist
linear_names = [n for n, m in model.named_modules() if isinstance(m, nn.Linear)]
print(f"Total Linear layers: {len(linear_names)}")
print(linear_names[:10])  # print a few to confirm naming

# --- Updated target names for Gemma architecture
TARGET_LINEAR_NAMES = {
    "self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj",
    "mlp.gate_proj", "mlp.up_proj", "mlp.down_proj"
}

# --- Re-inject LoRA with corrected matching
def _inject_lora_gemma(module: nn.Module, prefix: str = ""):
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(tn in qname for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora_gemma(child, qname)

_inject_lora_gemma(model)

# --- Verify LoRA successfully injected
print("Injected LoRA layers:", count_lora_layers(model))

# --- Freeze and verify trainables
lora.mark_only_lora_as_trainable(model)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M")

assert trainable > 0, "❌ No LoRA layers are trainable. Injection failed."


# ---- add this block ----
def count_lora_layers(m: nn.Module):
    n = 0
    for name, mod in m.named_modules():
        if isinstance(mod, lora.Linear):
            n += 1
    return n
# -------------------------

print("Injected LoRA layers:", count_lora_layers(model))


# freeze non-LoRA parameters
lora.mark_only_lora_as_trainable(model)

# Manually move the model to GPU *after* LoRA injection.
# This ensures the model is on the correct device before Trainer takes over.
if torch.cuda.is_available():
    model.to("cuda")

# sanity: how many params are trainable?
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M ({100*trainable/total:.2f}%)")

# --------------------
# Data: format Dolly into simple prompt-turns (no chat_template needed)
# --------------------
def _to_str(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    elif isinstance(x, str):
        return x
    return str(x) if x is not None else ""

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()

    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx

    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL requires list of strings

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

# --------------------
# Trainer (SFTTrainer)
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    args=training_args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save: only LoRA parameters
# --------------------
LORA_ADAPTER_PATH = os.path.join(OUTPUT_DIR, "loralib_adapter.pt")
torch.save(lora.lora_state_dict(model), LORA_ADAPTER_PATH)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", LORA_ADAPTER_PATH)
x

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Total Linear layers: 183
['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.0.mlp.gate_proj', 'model.layers.0.mlp.up_proj', 'model.layers.0.mlp.down_proj', 'model.layers.1.self_attn.q_proj', 'model.layers.1.self_attn.k_proj', 'model.layers.1.self_attn.v_proj']
Injected LoRA layers: 182
Trainable params: 20.77M / Total: 2635.11M
Injected LoRA layers: 182


OutOfMemoryError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 

In [ ]:
import os, math, torch, loralib as lora
import torch.nn as nn
from typing import Dict, Any, List
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments


# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"           # fits on Colab Pro GPU with small batch/accum
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-loralib-dolly"
RANK         = 16                             # LoRA rank
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()


os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Load tokenizer & base model (no quantization)
# --------------------
# --------------------
# Load tokenizer & base model (4-bit quantized)
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    device_map="auto",
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

model.config.attn_implementation = "flash_attention_2"


torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# -------------------


# --------------------
# Inject LoRA (loralib) by replacing target Linear layers
# --------------------
def _wrap_linear_with_lora(parent: nn.Module, child_name: str, child: nn.Module):
    # Skip meta tensors (lazy-loaded weights that aren't materialized yet)
    if child.weight.device.type == "meta":
        print(f"⚠️ Skipping {child_name}: still on meta device.")
        return

    in_f, out_f = child.in_features, child.out_features
    new_layer = lora.Linear(
        in_features=in_f,
        out_features=out_f,
        r=RANK,
        bias=(child.bias is not None)
    )
    # Copy pretrained weights to preserve original functionality
    with torch.no_grad():
        new_layer.weight.copy_(child.weight.data)
        if child.bias is not None:
            new_layer.bias.copy_(child.bias.data)

    setattr(parent, child_name, new_layer)
TARGET_LINEAR_NAMES = {
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
}

def _inject_lora(module: nn.Module, prefix: str = ""):
    # For each child, if it's a Linear whose qualified name ends with any of TARGET_LINEAR_NAMES, wrap with lora.Linear
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(qname.endswith(tn) for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora(child, qname)

# actually inject
# actually inject
_inject_lora(model)

# ---- add this block ----
def count_lora_layers(m: nn.Module):
    n = 0
    for name, mod in m.named_modules():
        if isinstance(mod, lora.Linear):
            n += 1
    return n
# -------------------------

print("Injected LoRA layers:", count_lora_layers(model))



# freeze non-LoRA parameters
lora.mark_only_lora_as_trainable(model)

# sanity: how many params are trainable?
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M ({100*trainable/total:.2f}%)")

# --------------------
# Data: format Dolly into simple prompt-turns (no chat_template needed)
# --------------------
def _to_str(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    elif isinstance(x, str):
        return x
    return str(x) if x is not None else ""

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()

    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx

    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL requires list of strings

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

# --------------------
# Trainer (SFTTrainer)
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    args=training_args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save: only LoRA parameters
# --------------------
LORA_ADAPTER_PATH = os.path.join(OUTPUT_DIR, "loralib_adapter.pt")
torch.save(lora.lora_state_dict(model), LORA_ADAPTER_PATH)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", LORA_ADAPTER_PATH)


In [ ]:
import os, math, torch, loralib as lora
import torch.nn as nn
from typing import Dict, Any, List
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments


# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"           # fits on Colab Pro GPU with small batch/accum
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-loralib-dolly"
RANK         = 16                             # LoRA rank
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
USE_BF16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Load tokenizer & base model
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


# Load model without device_map="auto" initially to ensure all weights are materialized on CPU.
# This prevents layers from being on a 'meta' device, allowing loralib to inject properly.
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    # Removed device_map="auto" here
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# Using "eager" attention implementation as recommended by earlier stderr for Gemma2 models
model.config.attn_implementation = "eager"

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# -------------------


# --------------------
# Inject LoRA (loralib) by replacing target Linear layers
# --------------------
def _wrap_linear_with_lora(parent: nn.Module, child_name: str, child: nn.Module):
    # This check is now less likely to trigger if model is fully loaded to CPU
    if child.weight.device.type == "meta":
        print(f"⚠️ Skipping {child_name}: still on meta device.")
        return

    in_f, out_f = child.in_features, child.out_features
    new_layer = lora.Linear(
        in_features=in_f,
        out_features=out_f,
        r=RANK,
        bias=(child.bias is not None)
    )
    # Copy pretrained weights to preserve original functionality
    with torch.no_grad():
        new_layer.weight.copy_(child.weight.data)
        if child.bias is not None:
            new_layer.bias.copy_(child.bias.data)

    setattr(parent, child_name, new_layer)
TARGET_LINEAR_NAMES = {
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
}

def _inject_lora(module: nn.Module, prefix: str = ""):
    # For each child, if it's a Linear whose qualified name ends with any of TARGET_LINEAR_NAMES, wrap with lora.Linear
    for name, child in list(module.named_children()):
        qname = f"{prefix}.{name}" if prefix else name
        if isinstance(child, nn.Linear) and any(qname.endswith(tn) for tn in TARGET_LINEAR_NAMES):
            _wrap_linear_with_lora(module, name, child)
        else:
            _inject_lora(child, qname)

# actually inject
_inject_lora(model)

# ---- add this block ----
def count_lora_layers(m: nn.Module):
    n = 0
    for name, mod in m.named_modules():
        if isinstance(mod, lora.Linear):
            n += 1
    return n
# -------------------------

print("Injected LoRA layers:", count_lora_layers(model))


# freeze non-LoRA parameters
lora.mark_only_lora_as_trainable(model)

# Manually move the model to GPU *after* LoRA injection.
# This ensures the model is on the correct device before Trainer takes over.
if torch.cuda.is_available():
    model.to("cuda")

# sanity: how many params are trainable?
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / Total: {total/1e6:.2f}M ({100*trainable/total:.2f}%")

# --------------------
# Data: format Dolly into simple prompt-turns (no chat_template needed)
# --------------------
def _to_str(x):
    if isinstance(x, list):
        return " ".join(map(str, x))
    elif isinstance(x, str):
        return x
    return str(x) if x is not None else ""

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()

    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx

    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL requires list of strings

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"]

# --------------------
# Trainer (SFTTrainer)
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    args=training_args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save: only LoRA parameters
# --------------------
LORA_ADAPTER_PATH = os.path.join(OUTPUT_DIR, "loralib_adapter.pt")
torch.save(lora.lora_state_dict(model), LORA_ADAPTER_PATH)
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved LoRA adapter to:", LORA_ADAPTER_PATH)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v_proj: still on meta device.
⚠️ Skipping o_proj: still on meta device.
⚠️ Skipping gate_proj: still on meta device.
⚠️ Skipping up_proj: still on meta device.
⚠️ Skipping down_proj: still on meta device.
⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v_proj: still on meta device.
⚠️ Skipping o_proj: still on meta device.
⚠️ Skipping gate_proj: still on meta device.
⚠️ Skipping up_proj: still on meta device.
⚠️ Skipping down_proj: still on meta device.
⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v_proj: still on meta device.
⚠️ Skipping o_proj: still on meta device.
⚠️ Skipping gate_proj: still on meta device.
⚠️ Skipping up_proj: still on meta device.
⚠️ Skipping down_proj: still on meta device.
⚠️ Skipping q_proj: still on meta device.
⚠️ Skipping k_proj: still on meta device.
⚠️ Skipping v

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

Pyrhon 3.11

In [ ]:
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu121
!pip install transformers==4.44.0 datasets==2.19.0 accelerate==0.34.0 peft==0.10.0 trl==0.9.6
!pip install bitsandbytes==0.43.1
!pip install numpy==1.26.4 scipy==1.11.4 sentencepiece protobuf==4.25.3



In [ ]:
import os, torch
from typing import Dict, Any, List
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# === Config ===
MODEL_NAME = "google/gemma-2-2b-it"  # Safer for T4. You can switch to "google/gemma-2-2b" after it's stable
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2-lora-dolly"
MAX_LEN      = 512          # start safe on T4. Try 768/1024 later.
EPOCHS       = 1.0
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
RANK         = 16
USE_BF16     = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Tokenizer ===
tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# === 4-bit quantization (bnb) ===
if not torch.cuda.is_available():
    raise SystemExit("❌ GPU not found. Switch Colab runtime to GPU (T4/L4/A100).")

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_cfg,
    device_map="auto",
)
base.config.pad_token_id = tok.pad_token_id
base.config.use_cache = False
# Try FA2 if available on your GPU stack
try:
    base.config.attn_implementation = "flash_attention_2"
except Exception:
    pass

# === LoRA via PEFT (no loralib) ===
lora_cfg = LoraConfig(
    r=RANK,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
model = get_peft_model(base, lora_cfg)
model.print_trainable_parameters()  # sanity check

# === Dolly formatting ===
def _to_str(x):
    if isinstance(x, list): return " ".join(map(str, x))
    if isinstance(x, str):  return x
    return "" if x is None else str(x)

def format_dolly(ex: Dict[str, Any]) -> List[str]:
    instr = _to_str(ex.get("instruction", "")).strip()
    ctx   = _to_str(ex.get("context", "")).strip()
    resp  = _to_str(ex.get("response", "")).strip()
    prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    if not prompt:
        prompt = "You are a helpful assistant."
    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return [text]  # TRL expects list[str]

ds = load_dataset(DATASET_NAME)
train_ds = ds["train"]

# === Trainer ===
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,  # memory saver
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tok,
    train_dataset=train_ds,
    args=args,
    formatting_func=format_dolly,
    max_seq_length=MAX_LEN,
    packing=False,
)

# quick preview
print("Preview:", format_dolly(train_ds[0])[0][:160].replace("\n"," ⏎ "), "...")

trainer.train()

# === Save adapters only ===
adapter_dir = os.path.join(OUTPUT_DIR, "lora_adapter")
trainer.model.save_pretrained(adapter_dir)
tok.save_pretrained(OUTPUT_DIR)
print("✅ Saved LoRA adapter to:", adapter_dir)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

QLORA

In [ ]:
# ==========================================================
# Gemma-2-2B QLoRA fine-tuning on Dolly-15k  (TRL ≥ 0.25)
# Works on Python 3.12 + PyTorch 2.4 + Transformers ≥ 4.45
# ==========================================================
import os, torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# --------------------
# Config
# --------------------
MODEL_NAME   = "google/gemma-2-2b"
DATASET_NAME = "databricks/databricks-dolly-15k"
OUTPUT_DIR   = "/content/outputs/gemma2b-qlora-dolly"
MAX_LEN      = 768
EPOCHS       = 1.5
BATCH_SIZE   = 1
GRAD_ACC     = 8
LR           = 2e-4
RANK         = 16
LORA_ALPHA   = 32
LORA_DROPOUT = 0.05
USE_BF16     = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------
# Tokenizer
# --------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------------------
# Quantization (bitsandbytes)
# --------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
)

# --------------------
# Base model
# --------------------
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16 if USE_BF16 else torch.float16,
    device_map="auto",
)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.attn_implementation = "eager"
model.gradient_checkpointing_enable()
prepare_model_for_kbit_training(model)

# --------------------
# PEFT (LoRA)
# --------------------
target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
]
peft_config = LoraConfig(
    r=RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config)

# --------------------
# Dataset preprocessing
# --------------------
def format_dolly(ex):
    instr = (ex.get("instruction") or "").strip()
    ctx   = (ex.get("context") or "").strip()
    resp  = (ex.get("response") or "").strip()
    prompt = "You are a helpful assistant."
    if instr:
        prompt = instr if not ctx else f"{instr}\n\nContext: {ctx}"
    elif ctx:
        prompt = ctx
    text = (
        f"<start_of_turn>user\n{prompt}\n<end_of_turn>\n"
        f"<start_of_turn>assistant\n{resp}\n<end_of_turn>"
    )
    return {"text": text}

dataset = load_dataset(DATASET_NAME)
train_data = dataset["train"].map(format_dolly, remove_columns=dataset["train"].column_names)

# Tokenize
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length",
    )

tokenized_train = train_data.map(tokenize, batched=True, remove_columns=["text"])

# --------------------
# TrainingArguments
# --------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC,
    learning_rate=LR,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=2,
    bf16=USE_BF16,
    fp16=(not USE_BF16),
    gradient_checkpointing=True,
    report_to="none",
)

# --------------------
# Trainer (TRL 0.25+)
# --------------------
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
)

# --------------------
# Train
# --------------------
trainer.train()

if torch.cuda.is_available():
    torch.cuda.synchronize()
    print("Max GPU memory (GB):", torch.cuda.max_memory_allocated() / (1024**3))

# --------------------
# Save Adapter + Tokenizer
# --------------------
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ QLoRA adapter + tokenizer saved to:", OUTPUT_DIR)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/15011 [00:00<?, ? examples/s]

Step,Training Loss
10,18.514700
20,3.608800
30,2.520100
40,1.513500
50,1.115100
60,0.999400
70,0.791900
80,0.758500


Step,Training Loss
10,18.514700
20,3.608800
30,2.520100
40,1.513500
50,1.115100
60,0.999400
70,0.791900
80,0.758500
90,1.494500
100,0.848500


# Text to image stable diffusion model

## Dependencies

In [ ]:
!pip uninstall -y sentence-transformers gcsfs
!pip install -q diffusers==0.27.2 transformers==4.40.1 accelerate==0.29.3 peft==0.10.0 bitsandbytes>=0.45.0 datasets==2.19.0 torchvision huggingface_hub==0.25.2

### Restart Runtime after install


### Change Param 'Use_QLora' in configuration to decide bnb_config to be Lora or QLora

In [ ]:
import time
import torch
import torch.nn.functional as F
import numpy as np
from accelerate import Accelerator
from datasets import load_dataset
from torchvision import transforms
from diffusers import DDPMScheduler, UNet2DConditionModel, AutoencoderKL
from transformers import CLIPTextModel, CLIPTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# CONFIGURATION
USE_QLORA      = True   # <--- SET TO FALSE for Standard LoRA
MODEL_ID       = "runwayml/stable-diffusion-v1-5" # Base SD 1.5 model
DATASET_ID     = "alexnasa/vangogh"               # Van Gogh Dataset
OUTPUT_DIR     = "./sd-vangogh-lora" if not USE_QLORA else "./sd-vangogh-qlora"
RESOLUTION     = 512
TRAIN_BATCH    = 1
GRAD_ACCUM     = 4
LEARNING_RATE  = 1e-4
MAX_STEPS      = 100    # Short run for profiling (increase for real results)
CHECKPOINTING  = True


# PROFILING UTILITIES
class Profiler:
    def __init__(self):
        self.start_time = None
        self.step_times = []

    def start(self):
        torch.cuda.reset_peak_memory_stats()
        self.start_time = time.time()

    def step_end(self):
        self.step_times.append(time.time())

    def print_stats(self, step):
        # Calculate memory
        mem_alloc = torch.cuda.memory_allocated() / 1024**3
        mem_max   = torch.cuda.max_memory_allocated() / 1024**3
        mem_res   = torch.cuda.memory_reserved() / 1024**3

        # Calculate speed
        if len(self.step_times) > 1:
            avg_time = (self.step_times[-1] - self.step_times[0]) / len(self.step_times)
            speed = 1.0 / avg_time
        else:
            speed = 0.0

        print(f"[Step {step}] VRAM: {mem_max:.2f}GB (Peak) | Speed: {speed:.2f} it/s")
        return mem_max


# 1. SETUP & MODEL LOADING
accelerator = Accelerator(gradient_accumulation_steps=GRAD_ACCUM, mixed_precision="bf16")
profiler = Profiler()

# Load Scheduler & Tokenizer
noise_scheduler = DDPMScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")
tokenizer = CLIPTokenizer.from_pretrained(MODEL_ID, subfolder="tokenizer")

# Load VAE and Text Encoder (Frozen, usually loaded in 16-bit to save memory)
vae = AutoencoderKL.from_pretrained(MODEL_ID, subfolder="vae", torch_dtype=torch.bfloat16)
text_encoder = CLIPTextModel.from_pretrained(MODEL_ID, subfolder="text_encoder", torch_dtype=torch.bfloat16)
vae.requires_grad_(False)
text_encoder.requires_grad_(False)

# Load UNet (The core model we finetune)
print(f"Loading UNet... Mode: {'QLoRA (4-bit)' if USE_QLORA else 'LoRA (16-bit)'}")

if USE_QLORA:
    # QLoRA: Load in 4-bit NF4
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet", quantization_config=bnb_config)
    unet = prepare_model_for_kbit_training(unet) # Important for gradient checkpointing in 4-bit
else:
    # Standard LoRA: Load in Float16
    unet = UNet2DConditionModel.from_pretrained(MODEL_ID, subfolder="unet", torch_dtype=torch.bfloat16)

unet.enable_gradient_checkpointing() # Save memory


# 2. INJECT LORA ADAPTERS
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["to_k", "to_q", "to_v", "to_out.0"], # Standard SD targets
    lora_dropout=0.05,
    bias="none",
)

unet = get_peft_model(unet, lora_config)
unet.print_trainable_parameters()

# Move models to GPU (Accelerator handles UNet, we manually move frozen ones)
vae.to(accelerator.device)
text_encoder.to(accelerator.device)

# Optimizer
optimizer = torch.optim.AdamW(unet.parameters(), lr=LEARNING_RATE)


# 3. DATASET PREP
dataset = load_dataset(DATASET_ID, split="train")

# Transform: Image -> Tensor, Text -> Token IDs
def transform_fn(examples):
    # 1. Process Images
    images = [transforms.Resize((RESOLUTION, RESOLUTION))(img).convert("RGB") for img in examples["image"]]
    pixel_values = [transforms.ToTensor()(img) * 2.0 - 1.0 for img in images] # List of Tensors

    # 2. Process Captions
    inputs = tokenizer(
        examples["caption"],
        max_length=tokenizer.model_max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # 3. CRITICAL FIX: Return ONLY the tensors.
    # Do not return 'image' (PIL) or 'caption' (String) keys.
    return {
        "pixel_values": pixel_values,
        "input_ids": inputs.input_ids
    }

train_dataset = dataset.with_transform(transform_fn)

# Now the DataLoader will only see Tensors, resolving the TypeError
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH, shuffle=True)

# Prepare with Accelerator
unet, optimizer, train_dataloader = accelerator.prepare(unet, optimizer, train_dataloader)


# 4. TRAINING LOOP
print("\nStarting Training...")
profiler.start()

unet.train()
global_step = 0

while global_step < MAX_STEPS:
    for batch in train_dataloader:
        with accelerator.accumulate(unet):
            # A. Convert images to latents (using VAE)
            # We must cast to weight_dtype (fp16) because VAE is frozen in fp16
            latents = vae.encode(batch["pixel_values"].to(dtype=torch.bfloat16, device=accelerator.device)).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # B. Add Noise
            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],), device=latents.device).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # C. Get Text Embeddings
            encoder_hidden_states = text_encoder(batch["input_ids"].to(accelerator.device))[0]

            # D. Predict Noise (Forward Pass)
            # For QLoRA, input must match compute dtype (fp16), but weights are 4-bit
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            # E. Loss & Backward
            loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")
            accelerator.backward(loss)

            optimizer.step()
            optimizer.zero_grad()

        if accelerator.sync_gradients:
            global_step += 1
            profiler.step_end()

            if global_step % 10 == 0:
                peak_mem = profiler.print_stats(global_step)
                print(f"   Loss: {loss.item():.4f}")

            if global_step >= MAX_STEPS:
                break


# 5. SAVE ADAPTERS
if accelerator.is_main_process:
    unet = accelerator.unwrap_model(unet)
    unet.save_pretrained(OUTPUT_DIR)
    print(f"\nSaved LoRA adapters to {OUTPUT_DIR}")

Loading UNet... Mode: QLoRA (4-bit)
trainable params: 3,188,736 || all params: 862,709,700 || trainable%: 0.3696186561945461

Starting Training...
[Step 10] VRAM: 6.08GB (Peak) | Speed: 0.13 it/s
   Loss: 0.4792
[Step 20] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.3079
[Step 30] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.0065
[Step 40] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.2232
[Step 50] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.0169
[Step 60] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.0840
[Step 70] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.3208
[Step 80] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.2421
[Step 90] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.0134
[Step 100] VRAM: 6.08GB (Peak) | Speed: 0.12 it/s
   Loss: 0.1641

Saved LoRA adapters to ./sd-vangogh-qlora
